# VKT visualization
time gradient for brain structure <br>
date = October 17, 2020 <br>

In [1]:
import vtk
from vtk.util import numpy_support

In [2]:
import os
import numpy as np
import cv2
import sys
import matplotlib.pyplot as plt

In [3]:
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../utils'))

In [4]:
from SliceText import SliceText
from VTKSegmentationMask import VTKSegmentationMask
from VTKSegmentationActors import VTKSegmentationActors

# Prepare data folder

In [5]:
structure = "Brain"
path_data_t0 = os.path.join("../../Data/Test/Segmentation/t0/")#, structure)
path_data_t1 = os.path.join("../../Data/Test/Segmentation/t1 10-10-2020/")#,  structure)#3-15-16
path_data_t2 = os.path.join("../../Data/test/Segmentation/t1 18-10-2020/")#,  structure)#3-23-16
#path_data_t0_img = "../../Data/Test/Segmentation/png 18-10-2020/"#3-15-16
path_data_mask = [path_data_t0,
                 path_data_t1, 
                 path_data_t2]

In [6]:
[os.path.join("../../Data/Test/Segmentation/",x) for x in os.listdir("../../Data/Test/Segmentation/") if 't0' in x or 't1' in x]

['../../Data/Test/Segmentation/t0',
 '../../Data/Test/Segmentation/t1 10-10-2020',
 '../../Data/Test/Segmentation/t1 18-10-2020']

# Dicom Data

In [7]:
# convert filepath
path_data_t0_img = "../../Data/Test/Segmentation/png 18-10-2020/"
pngfiles_t0_img = [os.path.join(path_data_t0_img,x) for x in os.listdir(path_data_t0_img)]
pngfiles_t0_img = sorted(pngfiles_t0_img, key=lambda x: int(x.split('slice')[-1].split('.')[0]))

filePath_t0_img = vtk.vtkStringArray()
filePath_t0_img.SetNumberOfValues(len(pngfiles_t0_img))
for i in range(0,len(pngfiles_t0_img),1):
    filePath_t0_img.SetValue(i,pngfiles_t0_img[i])

# Read png t0
png_reader_img = vtk.vtkPNGReader()
png_reader_img.SetFileNames(filePath_t0_img)
png_reader_img.Update()

dicom = vtk.vtkImageSlice()
dicom.SetMapper(vtk.vtkImageSliceMapper())
dicom.GetMapper().SetSliceNumber(0)
dicom.GetMapper().SetInputConnection(png_reader_img.GetOutputPort())
dicom.GetProperty().SetOpacity(0.5)
slice_number = dicom.GetMapper().GetSliceNumber()
slice_number_max = dicom.GetMapper().GetSliceNumberMaxValue()

slice_number

0

# VTK Segmentation Mask Generation

In [8]:
path_data_mask

['../../Data/Test/Segmentation/t0/',
 '../../Data/Test/Segmentation/t1 10-10-2020/',
 '../../Data/test/Segmentation/t1 18-10-2020/']

In [9]:
vtkConverter = VTKSegmentationMask(path_data_mask, fill=False)
vtk_mask, vtk_contour = vtkConverter.generate()
vtk_mask, vtk_contour

([(vtkCommonDataModelPython.vtkImageData)0000009DF105A228,
  (vtkCommonDataModelPython.vtkImageData)0000009DF105AD08],
 [(vtkCommonDataModelPython.vtkImageData)0000009DF10AA7C8,
  (vtkCommonDataModelPython.vtkImageData)0000009DF10AA768,
  (vtkCommonDataModelPython.vtkImageData)0000009DF10AA828])

In [10]:
number_time_steps = vtkConverter.number_time_steps
bg_color = vtkConverter.bg_color

# VTK Segmentation Actors

In [11]:
actorsGenerator = VTKSegmentationActors(vtk_mask, vtk_contour)
actors_mask, actors_contour = actorsGenerator.UpdateActors()

# Rendering

In [12]:
# Slice status message
sliceText = SliceText(slice_number, slice_number_max)

In [13]:
def MoveSliceFoward(obj, event):
    current_slice = dicom.GetMapper().GetSliceNumber()
    MoveSlice(current_slice + 1)

def MoveSliceBackward(obj, event):
    current_slice = dicom.GetMapper().GetSliceNumber()
    MoveSlice(current_slice - 1)

def MoveSlice(new_slice):
    new_slice = max(min(dicom.GetMapper().GetSliceNumberMaxValue(), new_slice), dicom.GetMapper().GetSliceNumberMinValue())
    print('setting new slice', new_slice)
    dicom.GetMapper().SetSliceNumber(new_slice)
    slice_number = dicom.GetMapper().GetSliceNumber()
    #segmentation_actors.SetSliceNumber(slice_number)
    #print('setting new mask slice', segmentation_actors.GetSliceNumber())
    for idx in range(len(actors_contour)):
        actors_contour[idx].GetMapper().SetSliceNumber(slice_number)
    for idx in range(len(actors_mask)):
        actors_mask[idx].GetMapper().SetSliceNumber(slice_number)
    sliceText.SetInput(new_slice, dicom.GetMapper().GetSliceNumberMaxValue())
    window.Render()

In [14]:
# Create renderer
renderer = vtk.vtkRenderer()
renderer.SetBackground(*bg_color)
renderer.ResetCamera()
renderer.AddActor2D(sliceText.sliceTextActor)
renderer.AddViewProp(dicom)
for idx in range(len(actors_mask)):
    renderer.AddViewProp(actors_mask[idx])
for idx in range(len(actors_contour)):
    renderer.AddActor(actors_contour[idx])

# Create interactor (customized)
interactorStyle = vtk.vtkInteractorStyleImage()
interactorStyle.SetInteractionModeToImageSlicing()
    
# Create interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetInteractorStyle(interactorStyle)
interactorStyle.AddObserver("MouseWheelForwardEvent", MoveSliceFoward)
interactorStyle.AddObserver("MouseWheelBackwardEvent", MoveSliceBackward)

# Create renderwindow
window = vtk.vtkRenderWindow()
window.SetSize(500,500)
window.AddRenderer(renderer)
window.SetInteractor(interactor)

In [15]:
# Render composite. In default mode.
window.Render()
interactor.Start()

setting new slice 1
setting new slice 2
setting new slice 3
setting new slice 4
setting new slice 5
setting new slice 6
setting new slice 7
setting new slice 8
setting new slice 9
setting new slice 10
setting new slice 11
setting new slice 12
setting new slice 13
setting new slice 14
setting new slice 15
setting new slice 16
setting new slice 17
setting new slice 18
setting new slice 19
setting new slice 19
setting new slice 19
setting new slice 19
setting new slice 18
setting new slice 17
setting new slice 16
setting new slice 15
setting new slice 14
setting new slice 13
setting new slice 12
setting new slice 11
setting new slice 10
setting new slice 9
setting new slice 8
setting new slice 7
setting new slice 6
setting new slice 5
setting new slice 4
setting new slice 3
setting new slice 4
setting new slice 5
setting new slice 6
setting new slice 7
setting new slice 8
setting new slice 9
setting new slice 10
setting new slice 11
setting new slice 12
setting new slice 13
setting new sl